# Learning Objectives

- Illustrate how functions can be used as tools in a single-agent system
- Implement the ReAct paradigm for single-agent systems

# Setup

In [3]:
!pip install -q crewai==0.114.0 \
                langchain-community==0.3.19

In [6]:
import uuid
import os
from crewai import LLM, Agent, Task, Crew, Process
from crewai.tools import tool
from google.colab import userdata

In [7]:
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [8]:
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    api_key=os.environ["GROQ_API_KEY"],
    tool_choice="auto"
    )

# Functions as Tools

## Business Scenario

**Domain: E-commerce Retail**

**Business Problem:** Automating initial customer support interactions regarding order status, product information, returns, and escalations. The goal is to resolve common queries quickly using tools and escalate complex issues when necessary.

### To be ignored

In this notebook, we implement a conversational AI agent designed for e-commerce customer support using the LangGraph library. The agent is built as a state machine (a graph) that can interact with a user, understand their request, and utilize a set of predefined "tools" to perform specific tasks like checking order status, product inventory, initiating returns, or escalating complex issues.

The core idea is to leverage a LLM to understand the user's intent and decide which tool (if any) is needed to fulfill the request. Note that we have built similar workflows before (the router pattern). The difference here is that the decision on which tool to pick and execute is autonomously taken by the reasoning capabilities of the agent.

## Approach

The agents we build are based on the ReAct (Reason + Act) prompting strategy. ReAct allows an LLM to work through problems step-by-step: it thinks about what to do next ("Thought"), chooses an action, often using a tool ("Action"), sees the result ("Observation"), and repeats this cycle until it finds the final answer.

We will implement a single-agent Crew designed to act as an E-commerce Customer Support Specialist. **Note that Crew AI uses pre-built ReAct agents as the default during instantiation of the `Agent` class**.

The implementation uses CrewAI's core components:

- **Tool Definition:** Four distinct tools (`get_order_details`, `check_product_inventory`, `initiate_return`, `escalate_issue`) are defined using CrewAI's `@tool` decorator. These functions encapsulate the specific actions the agent can perform, simulating interactions with backend e-commerce systems.

- **Agent Definition:** A single Agent (`support_agent`) is configured with a specific role (E-commerce Support Specialist), goal (resolve customer queries using tools), and backstory. These prompts guide the agent's LLM (`llm`) in reasoning about how to handle a query and whether to use its assigned tools. The tools are explicitly linked to this agent.

- **Task Definition:** A Task (`support_task`) is created with a detailed description that incorporates the user's query via a placeholder (`{customer_query}`). It instructs the agent on analyzing the query, selecting the appropriate tool, and formulating a response based on the tool's output or internal knowledge. An `expected_output` clarifies the desired result format.

- **Crew Orchestration:** A Crew object (`support_crew`) assembles the agent and task. It manages the execution flow (sequentially) and orchestrates the agent's reasoning, tool use, and response generation to accomplish the task based on the user input. Basic memory is implicitly handled per run, though more advanced memory could be configured if needed for multi-turn persistence beyond a single `kickoff`.

## Tool Definitions

In the code block below, we define each of the four core functions (`get_order_details`, `check_product_inventory`, `initiate_return`, `escalate_issue`).

- The decorator `@tool("tool_name")`: The crucial CrewAI decorator is applied to each function. The string argument (e.g., "get_order_details_tool") provides a unique name for the tool that the LLM will use.
- Docstrings: The function's docstring is vital. CrewAI passes this to the LLM as the tool's description, explaining its purpose, when to use it, and the expected input format (e.g., "Input must be the order ID string").
- Function Logic: Contains the simulated backend interaction logic (dictionary lookups, ID generation). print statements are kept for observing tool execution.
- `ecommerce_tools` List: Collects all decorated tool functions into a list for easy assignment to the agent.

In [9]:
@tool("get_order_details_tool")
def get_order_details(order_id: str) -> str:
    """
    Retrieves the status, items, and shipping address for a specific order ID.
    Use this tool ONLY when a customer asks about their order status or details.
    Input must be the order ID string (e.g., "ORD123").
    """
    print(f"--- Tool: get_order_details --- Input: {order_id=}")
    # --- Simulated Database Lookup ---
    orders_db = {
        "ORD123": {"status": "Shipped", "items": ["Laptop", "Mouse"], "address": "123 Main St", "tracking_id": "TRK456"},
        "ORD456": {"status": "Processing", "items": ["Keyboard"], "address": "456 Oak Ave", "tracking_id": None},
        "ORD789": {"status": "Delivered", "items": ["Monitor"], "address": "789 Pine Ln", "tracking_id": "TRK123"},
    }
    details = orders_db.get(order_id)
    if details:
        return f"Order {order_id} Status: {details['status']}. Items: {', '.join(details['items'])}. Shipping Address: {details['address']}. Tracking: {details.get('tracking_id', 'N/A')}"
    else:
        return f"Order ID '{order_id}' not found."

In [10]:
@tool("check_product_inventory_tool")
def check_product_inventory(sku: str) -> str:
    """
    Checks the current stock level for a specific product SKU.
    Use this tool ONLY when a customer asks about product availability or stock.
    Input must be the product SKU string (e.g., "LPTP001").
    """
    print(f"--- Tool: check_product_inventory --- Input: {sku=}")
    # --- Simulated Inventory Check ---
    inventory_db = {
        "LPTP001": 50,  # Laptop
        "MSE002": 120, # Mouse
        "KBD003": 0,   # Keyboard (Out of stock)
        "MON004": 35,  # Monitor
    }
    stock = inventory_db.get(sku)
    if stock is not None:
        availability = "In Stock" if stock > 0 else "Out of Stock"
        return f"Product SKU '{sku}' is currently {availability}. Quantity available: {stock}."
    else:
        return f"Product SKU '{sku}' not found in inventory system."

In [11]:
@tool("initiate_return_tool")
def initiate_return(order_id: str, sku: str, reason: str) -> str:
    """
    Initiates a return request for a specific item (SKU) from a given order ID, with a reason.
    Use this tool ONLY when a customer explicitly asks to return an item and provides the order ID,
    the product SKU, and a reason for the return.
    Inputs are: order_id (string), sku (string), reason (string).
    """
    # Note: CrewAI handles multiple arguments based on LLM understanding of the docstring/signature.
    print(f"--- Tool: initiate_return --- Input: {order_id=}, {sku=}, {reason=}")
    # --- Simulated Return Process ---
    return_id = f"RMA-{uuid.uuid4().hex[:6].upper()}"
    if order_id in {"ORD123", "ORD456", "ORD789"}: # Basic check
        return f"Return initiated for SKU '{sku}' from order '{order_id}'. Reason: '{reason}'. Your return ID is {return_id}. Instructions will be emailed shortly."
    else:
        return f"Could not initiate return. Order ID '{order_id}' not found."

In [12]:
@tool("escalate_issue_tool")
def escalate_issue(conversation_summary: str, reason_for_escalation: str) -> str:
    """
    Escalates the customer's issue to a human support agent when automated tools cannot resolve it.
    Use this tool ONLY if the customer's query is too complex, involves a complaint, requires special handling
    not covered by other tools, or if the customer explicitly requests human help after initial attempts fail.
    Inputs are: conversation_summary (string summarizing the issue), reason_for_escalation (string explaining why).
    """
    print(f"--- Tool: escalate_issue --- Input: {conversation_summary=}, {reason_for_escalation=}")
    # --- Simulated Escalation ---
    ticket_id = f"ESC-{uuid.uuid4().hex[:6].upper()}"
    print(f"\n*** Escalation Triggered ***")
    print(f"Summary: {conversation_summary}")
    print(f"Reason: {reason_for_escalation}")
    print(f"Ticket ID: {ticket_id}")
    print(f"***************************\n")
    return f"I understand this requires further assistance. I have escalated your issue to our support team. Your ticket ID is {ticket_id}. An agent will contact you shortly."

In [13]:
# List of tools for the agent
ecommerce_tools = [get_order_details, check_product_inventory, initiate_return, escalate_issue]

# Agent Definition

The code block below assembles the agent where an agent instance named `support_agent` is created.
- `role`, `goal`, `backstory`: These text prompts define the agent's persona and guide its LLM's reasoning. They are tailored to the e-commerce support specialist role, emphasizing tool usage and escalation procedures.
- `verbose=True`: Enables detailed logging of the agent's internal thought process, actions (including tool calls), and observations during execution.
- `allow_delegation=False`: Since it's a single-agent crew, delegation is disabled.
- `tools=ecommerce_tools`: Assigns the list of defined e-commerce tools to this agent, making them available for use.

In [14]:
support_agent = Agent(
  role='E-commerce Customer Support Specialist',
  goal='Accurately and efficiently resolve customer queries regarding orders, products, returns, or escalate issues when necessary, using the provided tools.',
  backstory=(
    "You are a friendly and highly capable AI customer support assistant for an online retailer. "
    "Your primary function is to help customers with common issues. "
    "You have access to tools for checking order status, product inventory, initiating returns, and escalating complex cases. "
    "Analyze the customer's query carefully. If it matches a tool's capability (like checking order 'ORD123' status, checking stock for 'KBD003', or returning 'MSE002' from order 'ORD123' due to 'defect'), use the appropriate tool. "
    "If the query is complex, requires information not available via tools, use the escalation tool. Provide clear and concise answers based on the tool results or inform the customer about the escalation."
  ),
  verbose=True, # Log the agent's thought process and actions
  allow_delegation=False, # This agent handles the tasks itself
  tools=ecommerce_tools, # Assign the defined e-commerce tools
  llm=llm # Use the pre-configured LLM
)

In [15]:
support_agent.model_dump()

/usr/local/lib/python3.12/dist-packages/pydantic/functional_validators.py:803: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `InstanceOf` - serialized value may not be as expected [field_name='agent_executor', input_value=<crewai.agents.crew_agent...bject at 0x7a90d3d2d700>, input_type=CrewAgentExecutor])
  function=lambda v, h: h(v), schema=original_schema


{'id': UUID('995d34d2-d94f-4b12-8db0-1c4e63331179'),
 'role': 'E-commerce Customer Support Specialist',
 'goal': 'Accurately and efficiently resolve customer queries regarding orders, products, returns, or escalate issues when necessary, using the provided tools.',
 'backstory': "You are a friendly and highly capable AI customer support assistant for an online retailer. Your primary function is to help customers with common issues. You have access to tools for checking order status, product inventory, initiating returns, and escalating complex cases. Analyze the customer's query carefully. If it matches a tool's capability (like checking order 'ORD123' status, checking stock for 'KBD003', or returning 'MSE002' from order 'ORD123' due to 'defect'), use the appropriate tool. If the query is complex, requires information not available via tools, use the escalation tool. Provide clear and concise answers based on the tool results or inform the customer about the escalation.",
 'cache': Tru

# Task Definition

The code block below creates a Task instance named `support_task`.
- `description`: Provides detailed, step-by-step instructions for the agent. It includes the placeholder {customer_query} which will be filled with the actual user input during kickoff. The description explicitly guides the agent on analyzing the query, selecting the correct tool based on the query type, extracting necessary arguments, and formulating a response. It reinforces the conditions for using each specific tool, including escalation.
- `expected_output`: Describes the desired final outcome - a clear, helpful response addressing the specific query, based on tool results or confirming escalation.
- `agent=support_agent`: Assigns this task specifically to the `support_agent`.

In [16]:
support_task = Task(
  description=(
    "Handle the following customer query: '{customer_query}'.\n"
    "1. Understand the customer's specific need (order status, product stock, return request, general question, or complaint).\n"
    "2. Based on the need, determine if one of the available tools is appropriate: "
    "   - Use 'get_order_details_tool' for order status/details requests (requires order ID).\n"
    "   - Use 'check_product_inventory_tool' for stock availability requests (requires SKU).\n"
    "   - Use 'initiate_return_tool' for return requests (requires order ID, SKU, reason).\n"
    "   - Use 'escalate_issue_tool' ONLY if the query cannot be resolved by other tools, is too complex.\n"
    "3. If a tool is appropriate, extract the necessary arguments (order_id, sku, reason, summary, etc.) from the query and execute the tool.\n"
    "4. Formulate a clear and helpful response to the customer based on the tool's output.\n"
    "5. If no tool is suitable or escalation is needed, explain briefly and use the escalate tool if appropriate, providing a summary and reason."
  ),
  expected_output=(
    "A final, direct, and helpful response to the customer addressing their query. "
    "This response should either contain the information requested (e.g., order status, stock level), "
    "confirm the action taken (e.g., return initiated with ID), or confirm escalation (with ticket ID)."
  ),
  agent=support_agent # Assign this task to our support agent
)

# Crew Assembly

Since this is a single-agent workflow, we have a crew of a single agent executing a single task.

In [17]:
support_crew = Crew(
  agents=[support_agent],
  tasks=[support_task],
  process=Process.sequential, # Tasks execute one after another (only one task here)
  verbose=True
)

# Execution

We can now execute the single-agent crew with a sample user_input string.

- An `inputs` dictionary maps the placeholder name in the task description (`customer_query`) to the user_input variable.

- `support_crew.kickoff(inputs=inputs)`: This is the command that starts the process. CrewAI takes the inputs, passes them to the `support_task`, activates the `support_agent`, and manages the internal loop of reasoning (LLM thinking), action (tool execution if decided), and observation (processing tool results) until the task's expected_output criteria are met.

In [18]:
user_input = "My order ORD456 has been processing for weeks and I can't track it. This is really frustrating!"

# Prepare the inputs for the task
inputs = {'customer_query': user_input}

# Start the crew's work
result = support_crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8b06482f-db4b-4f13-97e5-9dc82c82e580                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 268938ab-2bfb-411b-a6c4-c51466baed1c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 268938ab-2bfb-411b-a6c4-c51466baed1c
       Status: Executing Task...
    └── 🤖 Agent: E-commerce Customer Support Specialist
            Status: In Progress

# Agent: E-commerce Customer Support Specialist
## Task: Handle the following customer query: 'My order ORD456 has been processing for weeks and I can't track it. This is really frustrating!'.
1. Understand the customer's specific need (order status, product stock, return request, general question, or complaint).
2. Based on the need, determine if one of the available tools is appropriate:    - Use 'get_order_details_tool' for order status/details requests (requires order ID).
   - Use 'check_product_inventory_tool' for stock availability requests (requires SKU).
   - Use 'initiate_return_tool' for return requests (requires order ID, SKU, reason).
   - Use 'escalate_issue_tool' ONLY if the query cannot be resolved by other tools, is too complex.
3. If a tool is appropriate, extract the necessary arguments (order_id, sku, reason, summary, etc.) from the query and execute the tool.
4. Formulate a clear and helpful response to the customer based on the tool's output.
5. If no tool is suit

🤖 Agent: E-commerce Customer Support Specialist
    Status: In Progress
└── 🧠 Thinking...

...



# Agent: E-commerce Customer Support Specialist
## Final Answer: 
I've checked on the status of your order ORD456. It is currently being processed, and the items in your order include a keyboard. The shipping address for this order is 456 Oak Ave. However, it appears that there is no tracking information available at this time. If you have any further questions or concerns, please let me know and I'll be happy to help.




🚀 Crew: crew
└── 📋 Task: 268938ab-2bfb-411b-a6c4-c51466baed1c
       Status: Executing Task...
    └── 🤖 Agent: E-commerce Customer Support Specialist
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 268938ab-2bfb-411b-a6c4-c51466baed1c
       Assigned to: E-commerce Customer Support Specialist
       Status: ✅ Completed
    └── 🤖 Agent: E-commerce Customer Support Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 268938ab-2bfb-411b-a6c4-c51466baed1c                                                                     │
│  Agent: E-commerce Customer Support Specialist                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8b06482f-db4b-4f13-97e5-9dc82c82e580                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Let us inspect the above output to confirm that the ReAct pattern is followed.

1. Initial Context & Task Assignment:

    - The agent (E-commerce Customer Support Specialist) is assigned the task, which includes the user's query: 'My order ORD456 has been processing for weeks and I can't track it. This is really frustrating!'. This sets the stage and provides the initial information for the agent to reason about.

2. Reason/Thought Step:

    - Thought: `Thought: The customer is inquiring about the status of their order (ORD456) which has been processing for weeks. This is a request for order status/details, so I will use the 'get_order_details_tool' to retrieve the information about this order.`

    - Explanation: This section explicitly shows the Reasoning phase. The agent analyzes the user's query, identifies the core need (order status for ORD456), considers its available tools and their descriptions (implicitly referring back to the task description and tool docstrings), and decides on a plan: use the get_order_details_tool.

3. Act/Action Step:

    - Using tool: `get_order_details_tool`

    - Tool Input: `"{\"order_id\": \"ORD456\"}"`

    - Explanation: This corresponds to the Action phase. Having decided what to do in the reasoning step, the agent now executes that decision. It identifies the specific tool (`get_order_details_tool`) and prepares the necessary input (`{"order_id": "ORD456"}`) extracted from the query and its reasoning. CrewAI then invokes the actual Python function associated with this tool using this input.

4. Observe/Observation Step:

    - Tool Output: Order ORD456 Status: Processing. Items: Keyboard. Shipping Address: 456 Oak Ave. Tracking: None

    - Explanation: This is the Observation phase. The output shown here is the direct result returned by the executed `get_order_details` tool function. This new piece of information (the actual order status, items, address, and lack of tracking) is now available to the agent.

5. Subsequent Reasoning & Final Answer:

    - The agent implicitly enters another Reasoning phase after receiving the Tool Output (indicated by Thinking...). It analyzes the Observation ("Processing", "Tracking: None") in the context of the original query ("processing for weeks", "frustrating").

    - Final Answer: Your order ORD456 is currently processing. The items in your order include a keyboard, and it is being shipped to 456 Oak Ave. Unfortunately, there is no tracking information available at this time. I understand this is frustrating, and I recommend checking back soon for updates.

    - Explanation: In this final reasoning step, the agent determines that the information gathered from the tool is sufficient to answer the user's query. It synthesizes the tool's output ("Processing", items, address, no tracking) with empathy derived from the user's tone ("frustrating") to generate the final response. Since no further actions/tools are needed, it outputs the Final Answer, concluding the ReAct cycle for this task.

In summary: The verbose output clearly logs the agent's internal execution cycle, explicitly showing the "Thought" (Reasoning), the "Using tool" and "Tool Input" (Action), and the "Tool Output" (Observation). This loop allows the agent to dynamically interact with its tools to gather necessary information before formulating its final response, perfectly illustrating the ReAct paradigm in action.

In [19]:
# Print the final result from the crew's execution
print("\nFinal Answer:")
print(result)


Final Answer:
I've checked on the status of your order ORD456. It is currently being processed, and the items in your order include a keyboard. The shipping address for this order is 456 Oak Ave. However, it appears that there is no tracking information available at this time. If you have any further questions or concerns, please let me know and I'll be happy to help.


Let us look at another example.

In [20]:
user_input = "I want to return the mouse (MSE002) from order ORD123 because it's faulty."

# Prepare the inputs for the task
inputs = {'customer_query': user_input}

# Start the crew's work
result = support_crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8b06482f-db4b-4f13-97e5-9dc82c82e580                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 268938ab-2bfb-411b-a6c4-c51466baed1c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 268938ab-2bfb-411b-a6c4-c51466baed1c
       Status: Executing Task...
    └── 🤖 Agent: E-commerce Customer Support Specialist
            Status: In Progress

# Agent: E-commerce Customer Support Specialist
## Task: Handle the following customer query: 'I want to return the mouse (MSE002) from order ORD123 because it's faulty.'.
1. Understand the customer's specific need (order status, product stock, return request, general question, or complaint).
2. Based on the need, determine if one of the available tools is appropriate:    - Use 'get_order_details_tool' for order status/details requests (requires order ID).
   - Use 'check_product_inventory_tool' for stock availability requests (requires SKU).
   - Use 'initiate_return_tool' for return requests (requires order ID, SKU, reason).
   - Use 'escalate_issue_tool' ONLY if the query cannot be resolved by other tools, is too complex.
3. If a tool is appropriate, extract the necessary arguments (order_id, sku, reason, summary, etc.) from the query and execute the tool.
4. Formulate a clear and helpful response to the customer based on the tool's output.
5. If no tool is suitable or escalation is

🤖 Agent: E-commerce Customer Support Specialist
    Status: In Progress

...



# Agent: E-commerce Customer Support Specialist
## Final Answer: 
Return initiated for SKU 'MSE002' from order 'ORD123'. Reason: 'faulty'. Your return ID is RMA-9180FD. Instructions will be emailed shortly.




🚀 Crew: crew
└── 📋 Task: 268938ab-2bfb-411b-a6c4-c51466baed1c
       Status: Executing Task...
    └── 🤖 Agent: E-commerce Customer Support Specialist
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 268938ab-2bfb-411b-a6c4-c51466baed1c
       Assigned to: E-commerce Customer Support Specialist
       Status: ✅ Completed
    └── 🤖 Agent: E-commerce Customer Support Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 268938ab-2bfb-411b-a6c4-c51466baed1c                                                                     │
│  Agent: E-commerce Customer Support Specialist                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8b06482f-db4b-4f13-97e5-9dc82c82e580                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [21]:
# Print the final result from the crew's execution
print("\nFinal Answer:")
print(result)


Final Answer:
Return initiated for SKU 'MSE002' from order 'ORD123'. Reason: 'faulty'. Your return ID is RMA-9180FD. Instructions will be emailed shortly.


Another example in which user wants to return the product the never existed in the order

In [22]:
user_input = "I want to return the TV from order ORD123 because it's faulty."

# Prepare the inputs for the task
inputs = {'customer_query': user_input}

# Start the crew's work
result = support_crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8b06482f-db4b-4f13-97e5-9dc82c82e580                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 268938ab-2bfb-411b-a6c4-c51466baed1c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 268938ab-2bfb-411b-a6c4-c51466baed1c
       Status: Executing Task...
    └── 🤖 Agent: E-commerce Customer Support Specialist
            Status: In Progress

# Agent: E-commerce Customer Support Specialist
## Task: Handle the following customer query: 'I want to return the TV from order ORD123 because it's faulty.'.
1. Understand the customer's specific need (order status, product stock, return request, general question, or complaint).
2. Based on the need, determine if one of the available tools is appropriate:    - Use 'get_order_details_tool' for order status/details requests (requires order ID).
   - Use 'check_product_inventory_tool' for stock availability requests (requires SKU).
   - Use 'initiate_return_tool' for return requests (requires order ID, SKU, reason).
   - Use 'escalate_issue_tool' ONLY if the query cannot be resolved by other tools, is too complex.
3. If a tool is appropriate, extract the necessary arguments (order_id, sku, reason, summary, etc.) from the query and execute the tool.
4. Formulate a clear and helpful response to the customer based on the tool's output.
5. If no tool is suitable or escalation is needed, exp

🤖 Agent: E-commerce Customer Support Specialist
    Status: In Progress

...



# Agent: E-commerce Customer Support Specialist
## Final Answer: 
I understand this requires further assistance. I have escalated your issue to our support team. Your ticket ID is ESC-936EA4. An agent will contact you shortly.




🚀 Crew: crew
└── 📋 Task: 268938ab-2bfb-411b-a6c4-c51466baed1c
       Status: Executing Task...
    └── 🤖 Agent: E-commerce Customer Support Specialist
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 268938ab-2bfb-411b-a6c4-c51466baed1c
       Assigned to: E-commerce Customer Support Specialist
       Status: ✅ Completed
    └── 🤖 Agent: E-commerce Customer Support Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 268938ab-2bfb-411b-a6c4-c51466baed1c                                                                     │
│  Agent: E-commerce Customer Support Specialist                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 8b06482f-db4b-4f13-97e5-9dc82c82e580                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [23]:
# Print the final result from the crew's execution
print("\nFinal Answer:")
print(result)


Final Answer:
I understand this requires further assistance. I have escalated your issue to our support team. Your ticket ID is ESC-936EA4. An agent will contact you shortly.


<font size=6; color='blue'> **Happy Learning!** </font>
___